In [24]:
# Working through the example at
# https://github.com/Pyomo/pyomo/blob/master/pyomo/contrib/pynumero/examples/sensitivity.py
#
import pyomo.environ as pyo
from pyomo.contrib.pynumero.interfaces.pyomo_nlp import PyomoNLP
from pyomo.contrib.pynumero.sparse import BlockSymMatrix, BlockMatrix, BlockVector
from scipy.sparse import identity
from scipy.sparse.linalg import spsolve
import numpy as np

In [25]:
def create_model(eta1, eta2):
    model = pyo.ConcreteModel()
    # variables
    model.x1 = pyo.Var(initialize=0.15)
    model.x2 = pyo.Var(initialize=0.15)
    model.x3 = pyo.Var(initialize=0.0)
    # parameters
    model.eta1 = pyo.Var()
    model.eta2 = pyo.Var()

    model.nominal_eta1 = pyo.Param(initialize=eta1, mutable=True)
    model.nominal_eta2 = pyo.Param(initialize=eta2, mutable=True)

    # constraints + objective
    model.const1 = pyo.Constraint(expr=6*model.x1+3*model.x2+2*model.x3 - model.eta1 == 0)
    model.const2 = pyo.Constraint(expr=model.eta2*model.x1+model.x2-model.x3-1 == 0)
    model.cost = pyo.Objective(expr=model.x1**2 + model.x2**2 + model.x3**2)
    model.consteta1 = pyo.Constraint(expr=model.eta1 == model.nominal_eta1)
    model.consteta2 = pyo.Constraint(expr=model.eta2 == model.nominal_eta2)

    return model

def compute_init_lam(nlp, x=None, lam_max=1e3):
    if x is None:
        x = nlp.init_primals()
    else:
        assert x.size == nlp.n_primals()
    nlp.set_primals(x)

    assert nlp.n_ineq_constraints() == 0, "only supported for equality constrained nlps for now"

    nx = nlp.n_primals()
    nc = nlp.n_constraints()

    # create Jacobian
    jac = nlp.evaluate_jacobian()

    # create gradient of objective
    df = nlp.evaluate_grad_objective()

    # create KKT system
    kkt = BlockSymMatrix(2)
    kkt[0, 0] = identity(nx)
    kkt[1, 0] = jac

    zeros = np.zeros(nc)
    rhs = BlockVector([-df, zeros])

    flat_kkt = kkt.tocoo().tocsc()
    flat_rhs = rhs.flatten()

    sol = spsolve(flat_kkt, flat_rhs)
    return sol[nlp.n_primals() : nlp.n_primals() + nlp.n_constraints()]

#################################################################
m = create_model(4.5, 1.0)
opt = pyo.SolverFactory('ipopt')
results = opt.solve(m, tee=True)

#################################################################

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:       10
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0


In [26]:
# NLP solution for unperturbed problem
nlp = PyomoNLP(m)
x_unpert = nlp.init_primals()

print('Unperturbed NLP solution')
print(x_unpert)

Unperturbed NLP solution
[ 0.57653061  1.          0.37755102 -0.04591837  4.5       ]


In [27]:
nlp = PyomoNLP(m)
x = nlp.init_primals()
y = compute_init_lam(nlp, x=x)
nlp.set_primals(x)
nlp.set_duals(y)

J = nlp.evaluate_jacobian()
H = nlp.evaluate_hessian_lag()

M = BlockSymMatrix(2)
M[0, 0] = H
M[1, 0] = J

Np = BlockMatrix(2, 1)
Np[0, 0] = nlp.extract_submatrix_hessian_lag(pyomo_variables_rows=nlp.get_pyomo_variables(), pyomo_variables_cols=[m.eta1, m.eta2])
Np[1, 0] = nlp.extract_submatrix_jacobian(pyomo_variables=[m.eta1, m.eta2], pyomo_constraints=nlp.get_pyomo_constraints())

ds = spsolve(M.tocsc(), Np.tocsc())
print(nlp.variable_names())
print("ds:\n", ds.todense())

['x1', 'eta2', 'x2', 'x3', 'eta1']
ds:
 [[0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [28]:
#################################################################

p0 = np.array([pyo.value(m.nominal_eta1), pyo.value(m.nominal_eta2)])
p = np.array([4.45, 1.05])
dp = p - p0
dx = ds.dot(dp)[0:nlp.n_primals()]
new_x = x + dx
print("dp:", dp)
print("dx:", dx)
print("unperturbed NLP x:\n", x_unpert)
print("sensitivity based x:\n", new_x)

dp: [-0.05  0.05]
dx: [ 0.    0.05  0.    0.   -0.05]
unperturbed NLP x:
 [ 0.57653061  1.          0.37755102 -0.04591837  4.5       ]
sensitivity based x:
 [ 0.57653061  1.05        0.37755102 -0.04591837  4.45      ]


In [29]:
# somthing doesn't look right above since there is no change in x from unperturbed solution (only p changes)

#################################################################
m_pert = create_model(4.45, 1.05)
opt = pyo.SolverFactory('ipopt')
results = opt.solve(m_pert, tee=True)

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:       10
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0


In [30]:
nlp = PyomoNLP(m_pert)
x_pert = nlp.init_primals()
print("perturbed NLP x:\n", x_pert)
print("sensitivity based x:\n", new_x)

perturbed NLP x:
 [ 0.57101258  1.05        0.36495961 -0.03547717  4.45      ]
sensitivity based x:
 [ 0.57653061  1.05        0.37755102 -0.04591837  4.45      ]


In [31]:
# There seems to be a discrepancy between sensitivity based calculation and solving the perturbed NLP 
# based on sensitivity.py code. Below are 2 alternative modifications proposed that seem to fix the issue:
# option 1 - use submatrix of M, Np that has only variables x1, x2, x3 and constraints const1, const2. 
#            Also solve for -inv(M)*Np instead of inv(M)*Np
# option 2 - keep the same N matrix, but have Np as matrix of zeros and ones with identity for rows 
#            corresponding to paramters

In [32]:
# Code Modification: Option 1 - Use a submatrix of M without 
# the eta1, eta2 variables and consteta1, consteta2 constraints ---------------------


nlp = PyomoNLP(m)
x = nlp.init_primals()
y = compute_init_lam(nlp, x=x)
nlp.set_primals(x)
nlp.set_duals(y)

#J = nlp.evaluate_jacobian()
J = nlp.extract_submatrix_jacobian(pyomo_variables=[m.x1, m.x2, m.x3], pyomo_constraints=[m.const1, m.const2])

#H = nlp.evaluate_hessian_lag()
H = nlp.extract_submatrix_hessian_lag(pyomo_variables_rows=[m.x1, m.x2, m.x3], pyomo_variables_cols=[m.x1, m.x2, m.x3])

M = BlockSymMatrix(2)
M[0, 0] = H
M[1, 0] = J

Np = BlockMatrix(2, 1)
Np[0, 0] = nlp.extract_submatrix_hessian_lag(pyomo_variables_rows=[m.x1, m.x2, m.x3], pyomo_variables_cols=[m.eta1, m.eta2])
Np[1, 0] = nlp.extract_submatrix_jacobian(pyomo_variables=[m.eta1, m.eta2], pyomo_constraints=[m.const1, m.const2])

ds = spsolve(M.tocsc(), -Np.tocsc())
print(nlp.variable_names())
print("ds:\n", ds.todense())

['x1', 'eta2', 'x2', 'x3', 'eta1']
ds:
 [[ 0.1122449   0.00437318]
 [ 0.02040816 -0.23760933]
 [ 0.13265306  0.34329446]
 [-0.06122449 -0.04227405]
 [ 0.14285714  0.60204082]]


In [33]:
#################################################################
# x1, x2, x3 are variables 0, 2, 3 in nlp
p0 = np.array([pyo.value(m.nominal_eta1), pyo.value(m.nominal_eta2)])
p = np.array([4.45, 1.05])
dp = p - p0
dx = ds.dot(dp)[0:3]
new_x = [x[i] for i in [0, 2, 3]] + dx
print("dp:", dp)
print("dx:", dx)
print("perturbed NLP x:\n", [x_pert[i] for i in [0, 2, 3]])
print("sensitivity based x:\n", new_x)

dp: [-0.05  0.05]
dx: [-0.00539359 -0.01290087  0.01053207]
perturbed NLP x:
 [0.5710125845086471, 0.36495961309599173, -0.03547717316992889]
sensitivity based x:
 [ 0.57113703  0.36465015 -0.0353863 ]


In [35]:
#-------Code Modification: Option 2 - Modify Np to be matrix of zeros and ones ---------------------


nlp = PyomoNLP(m)
x = nlp.init_primals()
y = compute_init_lam(nlp, x=x)
nlp.set_primals(x)
nlp.set_duals(y)

J = nlp.evaluate_jacobian()
H = nlp.evaluate_hessian_lag()

M = BlockSymMatrix(2)
M[0, 0] = H
M[1, 0] = J

sens_vars = [m.eta1, m.eta2]
nsens  = len(sens_vars)
nr = M.shape[0]

sens_cons = ['consteta1', 'consteta2']
clist = nlp.constraint_names()
nc = len(clist)
Np = np.zeros((nr, nsens))
for i, cons in enumerate(sens_cons):
    Np[nr - nc + clist.index(cons), i] = 1

ds = spsolve(M.tocsc(), Np)
print(nlp.variable_names())
print("ds:\n", ds)

['x1', 'eta2', 'x2', 'x3', 'eta1']
ds:
 [[ 0.1122449   0.00437318]
 [ 0.          1.        ]
 [ 0.02040816 -0.23760933]
 [ 0.13265306  0.34329446]
 [ 1.         -0.        ]
 [ 0.14285714  0.60204082]
 [-0.06122449 -0.04227405]
 [-0.06122449 -0.04227405]
 [-0.04227405 -0.34553311]]


In [36]:
#################################################################

p0 = np.array([pyo.value(m.nominal_eta1), pyo.value(m.nominal_eta2)])
p = np.array([4.45, 1.05])
dp = p - p0
dx = ds.dot(dp)[0:nlp.n_primals()]
new_x = x + dx

print("dp:", dp)
print("dx:", dx)
print('new_x based on solving NLP at perturbed parameters')
print(x_pert)
print('new_x based on sensitivity calculation')
print(new_x)

dp: [-0.05  0.05]
dx: [-0.00539359  0.05       -0.01290087  0.01053207 -0.05      ]
new_x based on solving NLP at perturbed parameters
[ 0.57101258  1.05        0.36495961 -0.03547717  4.45      ]
new_x based on sensitivity calculation
[ 0.57113703  1.05        0.36465015 -0.0353863   4.45      ]


In [ ]:
# the sensitivity based x is now closer to the perturbed NLP x and results from option 1 and option 2 match